In [113]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from sklearn.metrics import r2_score
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import Imputer
from sklearn.linear_model import LassoCV,RidgeCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import PolynomialFeatures
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', 200)
import math

In [2]:
# Read-in the full data set
data = pd.read_csv('Final_Dataframe.csv')
data = data.drop(["Unnamed: 0"], axis=1)
data.head()

In [5]:
## train, test split##

# A train/test split is constructed where 90% of the subsample is 
# the train data set and 10% the test data set.

# Set train and test sizes
train_size = 0.9
test_size = 1-train_size

# Function to return random train and test sets
def data_splitter(df, train, validate=False, seed=9001):
    
    if validate:
        np.random.seed(seed)
        perm = np.random.permutation(df.index)
        m = len(df)
        train_end = int(train * m)
        validate_end = int(validate * m) + train_end
        train = df.ix[perm[:train_end]]
        validate = df.ix[perm[train_end:validate_end]]
        test = df.ix[perm[validate_end:]]
        return train, validate, test
    else:
        np.random.seed(seed)
        perm = np.random.permutation(df.index)
        m = len(df)
        train_end = int(train * m)
        train = df.ix[perm[:train_end]]
        test = df.ix[perm[train_end:]]
        return train, test

In [6]:
# Create train and test dataframes from subsample
train_df, test_df = data_splitter(data, train_size)

# Return shapes of train and test dataframes
print("Train Size: {}".format(train_df.shape))
print("Test Size: {}".format(test_df.shape))

Train Size: (1278, 949)
Test Size: (142, 949)


In [8]:
# Median imputation of missing values
imp = Imputer(missing_values='NaN', strategy='median', axis=1)
train_df = pd.DataFrame(imp.fit_transform(train_df), columns=data.columns)
test_df = pd.DataFrame(imp.transform(test_df), columns=data.columns)

train_df = train_df[train_df['Followers'] != 0]
test_df = test_df[test_df['Followers'] != 0]

# Final step: create y_train/x_train and y_test/x_test dataframes

# Initialize the training data
y_train = np.log(train_df['Followers'])
x_train = train_df.drop('Followers', axis=1)

# Initialize the testing data
y_test = np.log(test_df['Followers'])
x_test = test_df.drop('Followers', axis=1)

## Baseline models 

In this section, our goal is to examine the baseline performance of simple models on the test set. We would use these baseline test set r2 score as a reference for building more complex models. The models included in this section are mostly multilinear regression models with different subset of predictors and possible polynomial/interaction terms. PCA and LASSO/RRIDGE are explored here as well. 

We choose to start with linear regression because of high interpretability and low computational compexity. We can easily interpret the result of regression coeffcients in such fashion: holding all other predictors constant, one unit incrase in the specified predictor leads to a change in units indicated by the coefficent in the response variables. Linear regression also has a closed form solution for coefficients which reduces computational complexity.

Linear regression has the following assumptions: 
* There is a linear relationship between response varibles and predictors
* Residuals are independent
* Residuals are normally distributed
* Residuals has constant variance 

To evaluate baseline models, I use the metric r2. R2 is the proportion of overall variability of Y explained by the model. R2 has a caveat in the sense that it will always go up for the training set as we include more predictors. R2 will tend towards overfitting if we conduct model selection through R2. However, model selection in not the main goal 

### Part(a) Multilinear regression model with all predictors

As the first step, we fit a multilinear regression with all predictors that we have. This model would not neccessarily perform well since it has 949 predictors. However, we want to conduct a sanity check through this model to make sure that prediction power is reasonable.

The test r2 score for multilinear regression model with all predictors is -2.779. Our prediction does not perform well compared to the mean of response varaibles. This is evidence suggesting that we are overfitting our model with too many predictors. Therefore, going forward, we would like to fit a regression model with a subset of predictors in this model.

In [114]:
X=sm.add_constant(x_train)
X_test=sm.add_constant(x_test)
model=sm.OLS(y_train,X)
results=model.fit()
r2_test_a=r2_score(y_test,results.predict(X_test))
print("For multilinear regression with all terms,R2 score for training set: {}".format(r2_score(y_train,results.predict(X))))
print("For multilinear regression with all terms,R2 score for test set: {}".format(r2_test_a))
#results.summary()

For multilinear regression with all terms,R2 score for training set: 0.7655523815712503
For multilinear regression with all terms,R2 score for test set: -2.779712758436489


###  Part(b) Multilinear regression model with top artists predictors

In our prelimnary EDA anlaysis, we believed that top artists would be a good predictor for the success of a playlist. Therefore, here we fit two models in part (b) and (c) that only include predictors including artists. In part (b), we use the dummy variables of top 30 artists as predictors. As a note, for part (b), top artist are those who appear most often in playlists with 350,000+ followers. With more than 350,000 followers, a playlist will beat 80% of playlists in terms of followers. 

Our regression generates a test r2 score of 0.017, which is a lot better than -2.8 in part (a). Therefore, there is good reason to consider these predictors in future model building.

From the regression summary table, we get the list of significant top 30 artist predictors:
* 'Galantis', 'Post Malone', 'Yo Gotti', 'Ellie Goulding'

In [95]:
top_30_artist_col=['Lil Wayne', 'Van Morrison', 'Galantis',
       'Wiz Khalifa', 'Rihanna', 'Post Malone', 'Axwell /\ Ingrosso',
       'Young Thug', 'JAY Z', 'A$AP Rocky', 'Yo Gotti', 'Chance The Rapper',
       'Led Zeppelin', 'Otis Redding', '21 Savage', 'Deorro', 'Elton John',
       'SZA', 'Ty Dolla $ign', 'Ryan Adams', 'Birdy', 'Miguel', 'Niall Horan',
       'Ellie Goulding', 'Commodores', 'Radiohead', 'SYML', 'First Aid Kit',
       'Lord Huron']

x_train_art=x_train[top_30_artist_col]
x_test_art=x_test[top_30_artist_col]

X1=sm.add_constant(x_train_art)
X2=sm.add_constant(x_test_art)
model2=sm.OLS(y_train,X1)
results2=model2.fit()


With tio 30 artists,R2 score for test set: 0.017373740567596885


In [116]:
print("With top 30 artists,R2 score for training set: {}".format(r2_score(y_train,results2.predict(X1))))
print("With top 30 artists,R2 score for test set: {}".format(r2_score(y_test,results2.predict(X2))))
results2.summary()

With top 30 artists,R2 score for training set: 0.06172694503105225
With top 30 artists,R2 score for test set: 0.017373740567596885


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              Followers   R-squared:                       0.062
Model:                            OLS   Adj. R-squared:                  0.040
Method:                 Least Squares   F-statistic:                     2.784
Date:                Wed, 06 Dec 2017   Prob (F-statistic):           1.60e-06
Time:                        13:37:49   Log-Likelihood:                -3134.4
No. Observations:                1257   AIC:                             6329.
Df Residuals:                    1227   BIC:                             6483.
Df Model:                          29                                         
Covariance Type:            nonrobust                                         
======================================================================================
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
const                  9.5080      0.091    104.136      0.000       9.329       9.687
Lil Wayne              0.8115      0.828      0.980      0.327      -0.813       2.436
Van Morrison           0.0322      0.833      0.039      0.969      -1.603       1.667
Galantis               3.3362      1.093      3.051      0.002       1.191       5.481
Wiz Khalifa            0.3097      0.889      0.348      0.728      -1.435       2.054
Rihanna                0.5662      0.777      0.728      0.467      -0.959       2.091
Post Malone            2.4444      1.041      2.349      0.019       0.402       4.486
Axwell /\ Ingrosso     2.0688      1.121      1.846      0.065      -0.130       4.267
Young Thug            -1.0316      0.937     -1.101      0.271      -2.870       0.806
JAY Z                  0.6612      0.756      0.875      0.382      -0.822       2.144
A$AP Rocky             0.7276      0.969      0.751      0.453      -1.174       2.629
Yo Gotti               3.6842      1.057      3.485      0.001       1.610       5.758
Chance The Rapper      1.9356      1.074      1.801      0.072      -0.172       4.043
Led Zeppelin           1.7855      0.925      1.931      0.054      -0.029       3.600
Otis Redding           0.3608      0.989      0.365      0.715      -1.580       2.301
21 Savage              2.1001      1.173      1.790      0.074      -0.202       4.402
Deorro                 1.9751      1.074      1.840      0.066      -0.131       4.081
Elton John             1.2259      0.875      1.401      0.161      -0.491       2.943
SZA                    0.9203      0.914      1.007      0.314      -0.873       2.713
Ty Dolla $ign          0.7080      0.955      0.741      0.459      -1.166       2.582
Ryan Adams             0.8033      0.744      1.080      0.280      -0.656       2.262
Birdy                  0.5316      1.076      0.494      0.621      -1.579       2.642
Miguel                 1.0180      1.024      0.994      0.320      -0.991       3.027
Niall Horan            1.3225      0.952      1.390      0.165      -0.545       3.190
Ellie Goulding         1.7334      0.803      2.159      0.031       0.158       3.309
Commodores             0.9663      1.004      0.962      0.336      -1.004       2.937
Radiohead              0.2564      0.882      0.291      0.771      -1.474       1.987
SYML                   1.8656      1.176      1.586      0.113      -0.442       4.173
First Aid Kit          0.9468      0.958      0.988      0.323      -0.933       2.827
Lord Huron             0.5109      0.861      0.593      0.553      -1.179       2.201
==============================================================================
Omnibus:                       79.124   Durbin-Watson:                   1.955
Prob(Omnibus):                  0.000   Jarque-Bera (J

In [121]:
#significant artistis
results2.pvalues[results2.pvalues < 0.05].index[1:5]

Index(['Galantis', 'Post Malone', 'Yo Gotti', 'Ellie Goulding'], dtype='object')


###  Part(c) Multilinear regression model with top artists counts predictors

We continue to evaluate artist predictors in part (c). Here, top artists are defined differently from part (b).We first sum up the total number of followers for playlists that include an artist. Then, we rank the artists basing on the aggregated playlist followers. For part (c), the predictors are the number of top 10/10-20/20-30/30-40/40-50 artists that a playlist has.

From regression result, we see that r2 training result is 0.023 and test result is -0.03. The significant predictors include: "top_30_40", "top_40_50". It seems like part(c) r2 test score are lower than that in part(b), indicating that part(b) artist predictors has more power in predicting playlist followers thatn part(c) predictors. We should put more emphasis on part(b) predictors when constructing our best model.

In [122]:
x_train_art_count=x_train[top_artist_count_columns]
x_test_art_count=x_test[top_artist_count_columns]

X3=sm.add_constant(x_train_art_count)
X4=sm.add_constant(x_test_art_count)
model3=sm.OLS(y_train,X3)
results3=model3.fit()
#print("With top artists count predictors,R2 score for test set: {}".format(r2_score(y_test,results3.predict(X4))))
#results3.summary()

In [123]:
print("With top artists count predictors,R2 score for training set: {}".format(r2_score(y_train,results3.predict(X3))))
print("With top artists count predictors,R2 score for test set: {}".format(r2_score(y_test,results3.predict(X4))))
results3.summary()

With top artists count predictors,R2 score for training set: 0.012645473737053159
With top artists count predictors,R2 score for test set: -0.031235636069632644


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              Followers   R-squared:                       0.013
Model:                            OLS   Adj. R-squared:                  0.009
Method:                 Least Squares   F-statistic:                     3.204
Date:                Wed, 06 Dec 2017   Prob (F-statistic):            0.00702
Time:                        13:46:04   Log-Likelihood:                -3166.5
No. Observations:                1257   AIC:                             6345.
Df Residuals:                    1251   BIC:                             6376.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          9.6438      0.097     99.900      0.000       9.454       9.833
top_0_10       0.1075      0.274      0.392      0.695      -0.431       0.646
top_10_20      0.2819      0.358      0.788      0.431      -0.420       0.984
top_20_30     -0.1965      0.280     -0.702      0.483      -0.745       0.352
top_30_40      0.7624      0.273      2.789      0.005       0.226       1.299
top_40_50      0.7486      0.321      2.330      0.020       0.118       1.379
==============================================================================
Omnibus:                       79.718   Durbin-Watson:                   1.945
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               83.686
Skew:                          -0.595   Prob(JB):                     6.73e-19
Kurtosis:                       2.574   Cond. No.                         4.32
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

###  Part(d) Multilinear regression model with genre predictors

In our prelimnary EDA anlaysis, we also believed that genres would be a good predictor for playlist followers. Therefore, we fit a regression model with only genre predictors. Here，each predictors is a categorical variable indicating whether the playlist belongs to a specified genre.

When we use all of genre predictors, we see that training r2 score is 0.59 and test r2 score is -3.785. This again is the result of overfitting since we have 865 genre columns. Therefore, we fit another regression model with only significant genre predictors from the full genre regression model. This time, we have a training r2 score of 0.009 and test r2 score of 0.007. The number of significant genre predictors is 54.

Therefore, a subset of genre predictors could still be important and should be considered for building future models.

In [128]:
x_train_genre=x_train[genre_columns]
x_test_genre=x_test[genre_columns]

X5=sm.add_constant(x_train_genre)
X6=sm.add_constant(x_test_genre)
model4=sm.OLS(y_train,X5)
results4=model4.fit()

In [134]:
print("With all genre predictors,R2 score for training set: {}".format(r2_score(y_train,results4.predict(X5))))
print("With all genre predictors,R2 score for test set: {}".format(r2_score(y_test,results4.predict(X6))))
print("Number of genre predictors:{}".format(len(genre_columns)))
print("Significant genre predictors in regression:\n{}".format(results4.pvalues[results4.pvalues < 0.05]))

With all genre predictors,R2 score for training set: 0.5974648743813241
With all genre predictors,R2 score for test set: -3.7847086814421367
Number of genre predictors:865
Significant genre predictors in regression:
const                              1.678665e-87
 'alternative metal'               4.049398e-02
 'brooklyn indie'                  4.809165e-02
 'christian punk'                  1.321873e-02
 'classic rock'                    6.111137e-03
 'country christmas'               2.945994e-02
 'dirty south rap'                 4.232302e-02
 'dixieland'                       1.657556e-02
 'ectofolk'                        3.147615e-02
 'filter house'                    6.523992e-03
 'garage punk'                     4.496653e-02
 'garage rock'                     1.192386e-02
 'hauntology'                      3.862767e-02
 'indie garage rock'               1.106413e-02
 'indie psych-pop'                 4.345163e-02
 'industrial rock'                 3.083634e-02
 'industrial'   

In [132]:
sig_genre=results4.pvalues[results4.pvalues < 0.05].index[1:55]
x_train_genre=x_train[sig_genre]
x_test_genre=x_test[sig_genre]

X9=sm.add_constant(x_train_genre)
X10=sm.add_constant(x_test_genre)
model9=sm.OLS(y_train,X9)
results9=model9.fit()

In [133]:
print("With significant genre predictor,R2 score for training set: {}".format(r2_score(y_train,results9.predict(X9))))
print("With significant genre predictor,R2 score for test set: {}".format(r2_score(y_test,results9.predict(X10))))
print("Number of genre predictors:{}".format(len(sig_genre)))

With significant genre predictor,R2 score for training set: 0.09225710866655823
With significant genre predictor,R2 score for test set: 0.007097541941422314
Number of genre predictors:54


### Part (e) Multilinear regression with only siginifcant predictors from part (a)

Previously in part(a), our model includes all predictors and thus tend towards overfitting. In part(e), we fit a model with siginificant predictors from model in part(a) to reduce overfitting. We have a total of 49 predictors (cut down from 949 originally).

We see that test r2 score goes up to 0.085, which is the best r2 score so far. This indicates that our previous model indeed suffer from overfitting. We should work on choosing a subset of original predictors as predictors for furture models to improve prediction power.

In [136]:
print("Significant predictors from regression in part(a):\n{}".format(results.pvalues[results.pvalues < 0.05]))


Significant predictors from regression in part(a):
instrumentalness_mean              0.010220
liveness_mean                      0.001904
loudness_std                       0.038317
speech_mean                        0.021664
time_std                           0.005990
valence_mean                       0.000028
 'bass music'                      0.006956
 'big band'                        0.015642
 'christian punk'                  0.042034
 'country gospel'                  0.020285
 'crunk'                           0.026326
 'deep house'                      0.048536
 'dubstep'                         0.014862
 'ectofolk'                        0.016383
 'electro house'                   0.004962
 'escape room'                     0.043086
 'experimental'                    0.021957
 'filter house'                    0.032306
 'garage rock'                     0.011979
 'latin pop'                       0.031294
 'modern blues'                    0.022673
 'modern country rock'   

In [137]:
#fit a multilinear regression model with significant predictors
sig_preds=results.pvalues[results.pvalues < 0.05].index
x_train2 = x_train[sig_preds]
x_test2 = x_test[sig_preds]

X7=sm.add_constant(x_train2)
X8=sm.add_constant(x_test2)
model5=sm.OLS(y_train,X7)
results5=model5.fit()
r2_test_e=r2_score(y_test,results5.predict(X8))


In [140]:
print("With only the significant predictors in part(a),R2 score for test set: {}".format(r2_score(y_train, results5.predict(X7))))
print("With only the significant predictors in part(a),R2 score for test set: {}".format(r2_test_e))
results5.summary()

With only the significant predictors in part(a),R2 score for test set: 0.22290282653744364
With only the significant predictors in part(a),R2 score for test set: 0.0825894720034378


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              Followers   R-squared:                       0.223
Model:                            OLS   Adj. R-squared:                  0.191
Method:                 Least Squares   F-statistic:                     7.066
Date:                Wed, 06 Dec 2017   Prob (F-statistic):           1.25e-39
Time:                        14:06:25   Log-Likelihood:                -3016.0
No. Observations:                1257   AIC:                             6132.
Df Residuals:                    1207   BIC:                             6389.
Df Model:                          49                                         
Covariance Type:            nonrobust                                         
===================================================================================================
                                      coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------
const                              15.2241      0.950     16.027      0.000      13.361      17.088
instrumentalness_mean              -3.1447      1.089     -2.887      0.004      -5.282      -1.008
liveness_mean                     -12.1780      2.358     -5.164      0.000     -16.804      -7.552
loudness_std                        3.3418      1.299      2.572      0.010       0.793       5.891
speech_mean                        -2.5326      0.971     -2.608      0.009      -4.438      -0.627
time_std                         -1.25e-08   3.99e-09     -3.131      0.002   -2.03e-08   -4.67e-09
valence_mean                       -7.3241      1.413     -5.183      0.000     -10.096      -4.552
 'bass music'                      -0.4514      0.405     -1.116      0.265      -1.245       0.342
 'big band'                         0.2267      0.316      0.718      0.473      -0.393       0.847
 'christian punk'                  -1.1516      0.685     -1.682      0.093      -2.495       0.192
 'country gospel'                  -0.1673      0.326     -0.514      0.607      -0.806       0.471
 'crunk'                            1.4932      0.785      1.901      0.058      -0.048       3.034
 'deep house'                       0.3671      0.299      1.226      0.220      -0.220       0.954
 'dubstep'                          0.6460      0.540      1.196      0.232      -0.414       1.706
 'ectofolk'                        -2.1576      1.148     -1.880      0.060      -4.409       0.094
 'electro house'                   -0.2006      0.233     -0.859      0.390      -0.659       0.257
 'escape room'                      0.6030      0.182      3.308      0.001       0.245       0.961
 'experimental'                     0.1237      0.213      0.580      0.562      -0.295       0.543
 'filter house'                     1.0122      0.338      2.993      0.003       0.349       1.676
 'garage rock'                      0.2319      0.201      1.154      0.249      -0.163       0.626
 'latin pop'                        0.4265      0.220      1.942      0.052      -0.004       0.857
 'modern blues'                     0.5259      0.187      2.809      0.005       0.159       0.893
 'modern country rock'              0.4418      0.190      2.321      0.020       0.068       0.815
 'new orleans jazz'                -0.4096      0.363     -1.128      0.260      -1.122       0.303
 'pop emo'                         -0.1501      0.209     -0.719      0.472      -0.560       0.259
 'pop punk'                         0.4633      0.185      2.504      0.012       0.100       0.826
 'progressive electro house'        0.5527      0.332      1.666      0.096      -0.098       1.203
 'progressive house'               -0.3463      0.331     -1.048      0.295      -0

### Part (f) Bootstrapping for 10% Predictors

From previous parts, we observe that a smaller subset of original predictors may do a lot better in terms of test set prediction. Therefore, in part (f), we randomly choose 10% of predictors and fit a regression model. We do 500 iterations and record corresponding r2 test core and the associated predictors.

We achieve a r2 test score of 0.21. However,since we are just randomly choosing predictors, this result could come from chance alone and may not be very robust.

In [77]:
##bootstrapping for10% predictors
r2_test=[]
pred=[]
for i in range(500):
    train_col=[]
    while len(train_col)==0:
        for ele in x_train.columns:
            u=np.random.uniform(0,1)
            if u>0.9:
                if ele!='Followers':
                    train_col.append(ele)
    pred.append(train_col)
    x_train1 = x_train[train_col]
    x_test1 = x_test[train_col]
    multi2 =LinearRegression(fit_intercept=True)# no need to add constant when doing it this way
    multi2.fit(x_train1, y_train)
    r2_test.append(multi2.score(x_test1, y_test))

In [78]:
def findLargest(r2):
    largest=r2[0]
    count=0
    for i in range(len(r2)):
        if r2[i]>largest:
            largest=r2[i]
            count=i
    return count

In [142]:
print("After bootstrapping for 10% of predictors, the best R2 score for test set: {}".format(r2_test[findLargest(r2_test)]))
print("\n")
print('The assocaited predictors are:{}'.format(pred[findLargest(r2_test)]))


After bootstrapping for 10% of predictors, the best R2 score for test set: 0.21855216986291626


The assocaited predictors are:['acousticness_mean', 'liveness_std', 'loudness_mean', 'valence_mean', 'followers_std', 'top_0_10', " 'ambient'", " 'bebop'", " 'bluegrass'", " 'bow pop'", " 'chillhop'", " 'classic rock'", " 'classical piano'", " 'contemporary jazz'", " 'country dawn'", " 'deep talent show'", " 'desi hip hop'", " 'ethereal wave'", " 'freestyle'", " 'future garage'", " 'garage punk blues'", " 'heavy alternative'", " 'indie dream pop'", " 'indie garage rock'", " 'indie psych-rock'", " 'intelligent dance music'", " 'latin metal'", " 'metropopolis'", " 'modern uplift'", " 'motown'", " 'nashville sound'", " 'neo-industrial rock'", " 'new americana'", " 'noise rock'", " 'nu gaze'", " 'ok indie'", " 'piano blues'", " 'pop christmas'", " 'pop'", " 'power metal'", " 'progressive house'", " 'progressive post-hardcore'", " 'retro electro'", " 'scorecore'", " 'sludge metal'", " 'southern 

### Part(g) PCA

PCA (Principle Component Analysis) is another way to reduce the number of predictors. Each component is a linear combination of all 949 orginal predictors. The components are ordered in such a wa so that the amount of captured observed variance descends. In part(g), we implement PCA here. We try different numbers of PCA components from 1 to 100 and choose the optimal number of PCA components according to test r2 score.

We acheive the best r2 test score of 0.13 with 30 PCA components. Although we gain a higher test r2 score and also have less predictors, we lose a lot interpretability. We cannot pinpoint how change in one predictor will change the response varibable because each component is a linear combination of all original columns.

In [107]:
from sklearn.decomposition import PCA
r2_test_pca=[]
for i in range(1,100):
    pca = PCA(n_components=i)
    pca.fit(x_train)
    x_train_pca = pca.transform(x_train)
    x_test_pca = pca.transform(x_test)
    pca_regression_model = LinearRegression(fit_intercept=True)
    pca_regression_model.fit(x_train_pca, y_train)
    r2_test_pca.append(pca_regression_model.score(x_test_pca, y_test))

0.13027619329603601

In [109]:
print("After PCA, the best R2 score for test set: {}".format(r2_test_pca[findLargest(r2_test_pca)]))
print("The optimal number of components is: {}".format(findLargest(r2_test_pca)+1))

After PCA, the best R2 score for test set: 0.130276193296036
The optimal number of components is: 30


### Part(h) Lasso and Ridge

Lasso and Ridge regularizations are also methods to penalize overly complex models. To penalize coefficeints that has large magnitude, Lasso and ridge include the magnitude of the cofficients in the loss functions. Specific, Lasso includes the sum of the absolute values of coefficients multiplied by a constant lambda. Ridge includes the sum of the square of coefficients multiplied by a constant lambda.In part (h), we fit Ridge and Lasso with cross validation and with lamda ranging from 1e^-5 to 10^5.

With Lasso, the test r2 score is 0.109. With Ridge, the test r2 score is 0.112.

In [111]:
#lasso CV
lambda_list=[pow(10,i) for i in range(-5,5)]
lasso_regression = LassoCV(alphas=lambda_list, fit_intercept=True)
lasso_regression.fit(x_train, y_train)
lasso_regression.get_params
print("With lasso, R2 score for test set: {}".format(lasso_regression.score(x_test,y_test)))

With lasso, R2 score for test set: 0.1094129599174094


In [112]:
#ridge
ridge_regression = RidgeCV(cv=10,alphas=lambda_list, fit_intercept=True, normalize=True)
ridge_regression.fit(x_train, y_train)
print("With ridge, R2 score for test set: {}".format(ridge_regression.score(x_test,y_test)))

With ridge, R2 score for test set: 0.11234514421712671


In [144]:
print("With lasso, R2 score for test set: {}".format(lasso_regression.score(x_test,y_test)))
print("With ridge, R2 score for test set: {}".format(ridge_regression.score(x_test,y_test)))

With lasso, R2 score for test set: 0.1094129599174094
With ridge, R2 score for test set: 0.11234514421712671


## Summary of Baseline Models

Since we have 949 predictors in our original data, we encounter the problem of overfitting when constructing regression models. Therefore, for basline models, we focus on selecting a subgroup of predictors that perform well in terms of r2 test score. By using only the significant predictors from the full model, test r2 score improves a lot (from -2.7 to 0.8). PCA gives us r2 score of 0.13. Lasso gives up 0.109 and Ridge gives us 0.11. In addition, we should note that iteratively choosing 10% of predictors gives us the best baseline r2 metric (0.21). However, this model is not robust and tend toward overfitting since we do not methodologically choose the predictors. Still, we could use 0.21 as a reference when evaluating our future models. In summary, the most robust and predictive baseline model is PCA with r2 socre of 0.13. But we may use 0.21 as baseline r2_test metric for the furture. In addition, we should consider including top 30 artists predictors since they alone gives us 0.03 r2 test metric. On the other hand, we should carefully select which genre predictors to include since there are 865 of them and would lead to overfitting.